# Bounding H<sup>2</sup>MM, with object oriented calls

Let's get our obligitory imports in order, and we'll load the 3 detector data as well.

In [1]:
import os
import numpy as np
from matplotlib import pyplot as plt

import H2MM_C as hm

# load the data
color3 = list()
times3 = list()

i = 0
with open('sample_data_3det.txt','r') as f:
    for line in f:
        if i % 2 == 0:
            times3.append(np.array([int(x) for x in line.split()],dtype='Q'))
        else:
            color3.append(np.array([int(x) for x in line.split()],dtype='L'))
        i += 1

## Built in Limits

Sometimes we may want to restrain what values are possible in the H<sup>2</sup>MM model, for instance to keep transition rates within reasonable values, or because you know something about how the emmission probability matrix (`obs`) should behave.

This is expected to happen most often (but not exclusively) when some experimental parameter is periodic, but not important to the data. For instance in $\mu$sALEX experiments, the laser alternation period causes donor and acceptor excitation photons, which arrive in separate streams to alternate (and thus transition) perfectly periodically, yet that has no bearing on transitions between our system. Thus transition rates close to the rate of laser alternation are likely to be artifacts, and thus we want to exclude them. For $\mu sALEX$ experiments, we find this still is not enough.

To define some bounds, we need to define the bounds, this is done using the `hm.h2mm_limits` object, which we pass into the `.optimize()` method through the keyword argument `bounds`, and we also need to supply a method string to another keyword argument, `bounds_func`.

#### Let's see an example

In [2]:
alt_period = 4000 # a fake alternation period
us_bounds = hm.h2mm_limits(max_trans = 1/(alt_period))

prior = np.array([1/4, 1/4, 1/4, 1/4])
trans = np.array([[1-3e-6, 1e-6, 1e-6, 1e-6],
                  [1e-6, 1-3e-6, 1e-6, 1e-6],
                  [1e-6, 1e-6, 1-3e-6, 1e-6],
                  [1e-6, 1e-6, 1e-6, 1-3e-6]])
obs = np.array([[0.4,0.4,0.2],
                [0.3,0.1,0.6],
                [0.2,0.4,0.4],
                [0.1,0.1,0.8]])

us_opt_model4 = hm.h2mm_model(prior, trans, obs)

us_opt_model4.optimize(color3, times3, bounds_func='revert', bounds=us_bounds)
us_opt_model4

The model converged after 631 iterations

nstate: 4, ndet: 3, nphot: 436084, niter: 631, loglik: -408203.01780807413 converged state: 3
prior:
0.19742522045703978, 0.5611254558625506, 0.24144932368040967, 7.251074733803432e-42
trans:
0.9999562426518485, 2.620839826060173e-05, 1.8189622724271526e-06, 1.572998761849734e-05
7.049720131796072e-06, 0.9999698856343252, 6.991342045372026e-06, 1.6073303497560515e-05
1.2716807355059964e-06, 1.738821760851245e-05, 0.9999781791003083, 3.1610013477364306e-06
1.730182323457899e-05, 0.00011452568669777748, 8.076641015599347e-06, 0.999860095849052
obs:
0.8495286641815059, 0.07564782657329697, 0.07482350924519719
0.4716858174332932, 0.09134399902467155, 0.43697018354203526
0.14909987819343537, 0.31276918990273284, 0.5381309319038319
0.15084679777173995, 0.07681315977150291, 0.7723400424567571

So, what did we just do? The `hm.h2mm_limits` object `us_bounds` prevents any value (off the diagonal) of the **transition probability** matrix (`.trans`) from ever being larger (i.e. faster transition rate) than `1/(4000)`. 

#### Bounds process

When you use a bounds method, each iteration goes through the following steps:
1. Calculate *loglikelihood* and new model
2. Check if the **model** converged
3. Analyze the **new model**, and correct if necessary
    1. Check if any values are smaller or larger than a pre-set minimum or maximum
    2. If values are out of bounds, apply correction, method defined by argument passed to `bounds_func`
4. Repeat optimization (back to step 1)

The inputs to `hm.h2mm_limits` are all keyword argumetns, and come in the form of `min/max_[array]` where `[array]` is `prior`, `trans` or `obs`, and specify the minimum and maximum values in the respective array.
Specifying as a float will set the value for all states, and thus the created `hm.h2mm_limits` object can be used for models with any model, while values can be specified as arrays, where each element sets the min/max of the value at that position in the given array of the model.

#### `bounds_func`

As mentioned in the above outline, the bounding process needs to choose how to correct the way in which a model value that is out of bound is corrected.
There are 3 options:

1. `minmax` shallowest correction, sets the value to its minimum or maximum
2. `revert` prefered method, sets the value to the value in the previous model
3. `revert_old` a more extreme form of `revert` which goes to the model before the last in the optimization, and sets the value to that.

### Using `hm.factory_h2mm_model()` with bounds

You will note in the previous example, we specified the `hm.h2mm_model` explicitly, instead of using `hm.factory_h2mm_model()`. 
This is because it is possible that the `hm.factory_h2mm_model()` could create an initial model that contains out of bounds values, which could result in odd behavior during optimization.

There is a way around this, you can give the `hm.h2mm_limits` object to `hm.factory_h2mm_model()` through the keyword argument `bounds`, and the function will automatically ensure the model is with bounds:

> See the full documentation to see full list of options for customizing the `hm.factory_h2mm_model()` function's output

In [3]:
us_bounds = hm.h2mm_limits(max_trans = 1/4000)
# make factory_h2mm_model make a model within bounds
us_model = hm.factory_h2mm_model(3,3, bounds=us_bounds)
us_model.optimize(color3, times3, bounds=us_bounds, bounds_func='revert')

The model converged after 198 iterations

nstate: 3, ndet: 3, nphot: 436084, niter: 198, loglik: -409379.1470199696 converged state: 3
prior:
0.25991591664148433, 0.4943777171497025, 0.2457063662088131
trans:
0.9999762009735097, 2.109177405070597e-05, 2.70725243958023e-06
8.392924952755617e-06, 0.9999812078566601, 1.0399218387024308e-05
6.289922278794732e-06, 4.466274697500153e-05, 0.9999490473307463
obs:
0.1457041699146636, 0.2934433052426973, 0.560852524842639
0.44173947516607204, 0.08763107972905039, 0.47062944510487764
0.8414290949399048, 0.07852868456586601, 0.08004222049422913

## Custom Bounds

In [4]:
def sample_bounds(new_model,current_model,old_model,bound):
    # it's usually best to just keep the function signature the same
    # grab the obs matrix
    obs = new_model.obs
    # set first row of obs matrix to bound
    obs[0,:] = bound
    # change the obs matrix of the new model
    new_model.obs = obs
    # return the adjusted model
    return new_model

In [5]:
bnd = np.array([0.09,0.01,0.9])
prior = np.array([1/4, 1/4, 1/4, 1/4])
trans = np.array([[1-3e-6, 1e-6, 1e-6, 1e-6],
                  [1e-6, 1-3e-6, 1e-6, 1e-6],
                  [1e-6, 1e-6, 1-3e-6, 1e-6],
                  [1e-6, 1e-6, 1e-6, 1-3e-6]])
obs = np.array([bnd,
                [0.3,0.1,0.6],
                [0.2,0.4,0.4],
                [0.1,0.1,0.8]])

us_opt_model4 = hm.h2mm_model(prior, trans, obs)
us_opt_model4.optimize(color3, times3, bounds_func=sample_bounds, bounds=bnd)

The model converged after 727 iterations

nstate: 4, ndet: 3, nphot: 436084, niter: 727, loglik: -408466.0068841426 converged state: 3
prior:
1.455755669727489e-187, 0.20970786415951875, 0.2488678925825922, 0.5414242432578891
trans:
0.9997618615612152, 1.7567290399074657e-05, 3.5821037770349356e-05, 0.00018475011061545534
8.503052104689647e-06, 0.9999540030420222, 3.205677542920319e-06, 3.428822833015612e-05
5.2378185734919716e-06, 1.647285650059047e-06, 0.9999755760932615, 1.7538802514998113e-05
1.3451082616098721e-05, 8.342115431210433e-06, 6.512280834898936e-06, 0.9999716945211178
obs:
0.09, 0.01, 0.9
0.8479463979984883, 0.07610555511419342, 0.07594804688731839
0.1488982237914183, 0.30755353820272413, 0.5435482380058576
0.46010303873879893, 0.09244656142114777, 0.4474503998400533